In [ ]:


from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob



In [ ]:
! mkdir pepper
! unzip "/content/drive/MyDrive/PEPPER IMAGE SET try 1.zip" -d pepper

mkdir: cannot create directory ‘pepper’: File exists
Archive:  /content/drive/MyDrive/PEPPER IMAGE SET try 1.zip
   creating: pepper/Resized/
   creating: pepper/Resized/BIG DOT/
  inflating: pepper/Resized/BIG DOT/IMG20210208170109.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208172638.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131635.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131514.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131511.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208170645.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208130354.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208172806.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210207112458.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208171735.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208124945.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208172408.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131002_01.jpg  
  inflating: pepper/Resized/BIG DOT/IM

In [ ]:
!pip install split-folders
import splitfolders

In [ ]:
splitfolders.ratio("/content/pepper/Resized", output="/content/dataset", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values

Copying files: 1416 files [00:00, 6120.48 files/s]


In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'datasets/train'
valid_path = 'dataset/test'

In [ ]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 0s 0us/step


In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
folders = glob('dataset/train/*')

In [ ]:
x = Flatten()(resnet.output) #add

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1131 images belonging to 7 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 146 images belonging to 7 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
36/36 [==============================] - 208s 6s/step - loss: 8.6244 - accuracy: 0.1840 - val_loss: 2.3518 - val_accuracy: 0.2808
Epoch 2/10
36/36 [==============================] - 204s 6s/step - loss: 2.3652 - accuracy: 0.2508 - val_loss: 1.9804 - val_accuracy: 0.2329
Epoch 3/10
36/36 [==============================] - 204s 6s/step - loss: 2.1078 - accuracy: 0.2621 - val_loss: 2.1245 - val_accuracy: 0.1918
Epoch 4/10
36/36 [==============================] - 204s 6s/step - loss: 2.2393 - accuracy: 0.2797 - val_loss: 1.8842 - val_accuracy: 0.3014
Epoch 5/10
36/36 [==============================] - 203s 6s/step - loss: 2.0287 - accuracy: 0.2901 - val_loss: 2.2857 - val_accuracy: 0.2329
Epoch 6/10
36/36 [==============================] - 203s 6s/step - loss: 1.9032 - accuracy: 0.2995 - val_loss: 2.1120 - val_accuracy: 0.2877
Epoch 7/10
36/36 [==============================] - 203s 6s/step - loss: 1.9229 - accuracy: 0.3551 - val_loss: 1.8865 - val_accuracy: 0.3904
Epoch 8/10
36

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_resnet50.h5')

In [ ]:
img_data

In [ ]:
img=image.load_img('Datasets/Test/Coffee/download (2).jpg',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x=x/255

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)